Empleando los datos del archivo './src/crypto_currency'.

- Cargue los datos en una base de datos apropiada.
- Lea mediante pandas las bases cargadas
- Emplee un metodo de concatenación para unificar la información.(debe considerar que se añadirá una columna adicional que especifique el tipo de moneda)
- Genere un agrupamiento de información a manera de obtener un resumen de los datos.
- Almacene dichos datos en un reporte excel.
- Apoyandose del ejercicio 2. Cree una imagen que sea guarda en el archivo excel
- Emplee un método de envio de correos



In [ ]:
import pandas as pd
import sqlite3

# Cargar datos
df_btc = pd.read_csv('./src/crypto_currency/btc_price.csv')
df_eth = pd.read_csv('./src/crypto_currency/eth_price.csv')

# Conectar a la base de datos SQLite
db_path = './src/crypto_currency/crypto_data.db'
conn = sqlite3.connect(db_path)

# Almacenar los DataFrames en la base de datos
df_btc.to_sql('btc_data', conn, index=False, if_exists='replace')
df_eth.to_sql('eth_data', conn, index=False, if_exists='replace')

# Cerrar la conexión
conn.close()


In [ ]:
# Leer desde la base de datos
conn = sqlite3.connect(db_path)

df_btc_read = pd.read_sql_query('SELECT * FROM btc_data', conn)
df_eth_read = pd.read_sql_query('SELECT * FROM eth_data', conn)

conn.close()

# Mostrar los primeros registros de cada DataFrame
print("BTC Data:")
print(df_btc_read.head())
print("\nETH Data:")
print(df_eth_read.head())

In [ ]:
# Añadir una columna adicional que especifique el tipo de moneda
df_btc_read['Currency'] = 'BTC'
df_eth_read['Currency'] = 'ETH'

# Concatenar los DataFrames
df_concatenated = pd.concat([df_btc_read, df_eth_read], ignore_index=True)

# Mostrar los primeros registros del DataFrame concatenado
print("Concatenated Data:")
print(df_concatenated.head())

In [ ]:
# Agrupar por tipo de moneda y obtener un resumen
df_summary = df_concatenated.groupby('Currency').agg({
    'Open': 'mean',
    'High': 'max',
    'Low': 'min',
    'Close': 'mean',
    'Volume': 'sum'
}).reset_index()

# Mostrar el resumen
print("Summary:")
print(df_summary)

In [ ]:
# Almacenar en un informe Excel
excel_report_path = './src/crypto_currency/crypto_report.xlsx'
with pd.ExcelWriter(excel_report_path, engine='xlsxwriter') as excel_writer:
    df_concatenated.to_excel(excel_writer, sheet_name='All_Data', index=False)
    df_summary.to_excel(excel_writer, sheet_name='Summary', index=False)

    # Generar y almacenar la imagen en el informe Excel
    plt.figure(figsize=(8, 6))
    plt.plot(df_concatenated['Date'], df_concatenated['Close'])
    plt.title('Closing Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.savefig('./src/crypto_currency/closing_prices_plot.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Insertar la imagen en el informe Excel
    image_path = './src/crypto_currency/closing_prices_plot.png'
    worksheet = excel_writer.sheets['All_Data']
    worksheet.insert_image('H2', image_path, {'x_scale': 0.7, 'y_scale': 0.7})

print(f"Excel report saved at: {excel_report_path}")


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email():
    sender_email = 'angelchumbimunisandoval@gmail.com'
    sender_password = 'teses pfkr spgp gqdm'
    receiver_email = 'angel.chumbimuni@unmsm.edu.pe'



    subject = 'Informe Crypto Currency'
    body = 'Adjunto encontrarás el informe sobre las criptomonedas.'

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Adjuntar el informe Excel al correo
    attachment = open(excel_report_path, 'rb')
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    part.add_header('Content-Disposition', 'attachment; filename="crypto_report.xlsx"')
    msg.attach(part)

    # Establecer conexión SMTP y enviar correo
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print("Correo electrónico enviado con éxito.")

# Llamamos a la función de envío de correo electrónico
send_email()
